In [2]:
import os, gc, zipfile, time
import numpy as np, cv2, tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# === UNZIP ===
# zipPath = "/content/dataset.zip"
# with zipfile.ZipFile(zipPath, 'r') as zip_ref:
#     zip_ref.extractall("/content")
datasetPath = "/content/dataset"

# === LOAD IMAGES ===
def loadImages(folderPath, label, imgSize=(128,128)):
    data, labels = [], []
    for file in os.listdir(folderPath):
        if file.lower().endswith(('.jpg','.png','.jpeg')):
            img = cv2.imread(os.path.join(folderPath,file))
            if img is None: continue
            img = cv2.resize(img, imgSize)
            data.append(img)
            labels.append(label)
    return np.array(data), np.array(labels)

def loadSplit(split):
    x, y = [], []
    splitPath = os.path.join(datasetPath, split)
    for className in os.listdir(splitPath):
        folderPath = os.path.join(splitPath, className)
        imgs, labels = loadImages(folderPath, className)
        x.append(imgs)
        y.append(labels)
    return np.concatenate(x), np.concatenate(y)

print("Loading data...")
XTrain, yTrain = loadSplit("train")
XValid, yValid = loadSplit("valid")
XTest, yTest = loadSplit("test")

print(f"Samples: train={len(XTrain)}, valid={len(XValid)}, test={len(XTest)}")

labelEncoder = LabelEncoder()
yTrainEnc = labelEncoder.fit_transform(yTrain)
yValidEnc = labelEncoder.transform(yValid)
yTestEnc = labelEncoder.transform(yTest)

# === MobileNetV2 base for feature extraction ===
baseModel = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))
featureExtractor = Sequential([
    baseModel,
    GlobalAveragePooling2D()
])
featureExtractor.trainable = False

def extractFeatures(model, data, batch_size=64):
    features = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        batchPre = preprocess_input(batch.astype(np.float32))
        feat = model.predict(batchPre, verbose=0)
        features.append(feat)
    return np.vstack(features)

print("Extracting train features...")
XTrainFeat = extractFeatures(featureExtractor, XTrain)
print("Extracting valid features...")
XValidFeat = extractFeatures(featureExtractor, XValid)
print("Extracting test features...")
XTestFeat = extractFeatures(featureExtractor, XTest)

gc.collect()

# === Classic ML on extracted features ===
start = time.time()
knn = KNeighborsClassifier(n_jobs=-1)
knn.fit(XTrainFeat, yTrainEnc)
knnPred = knn.predict(XTestFeat)
print(f"KNN Accuracy: {accuracy_score(yTestEnc, knnPred):.4f}")
print(f"KNN Time: {time.time() - start:.2f} sec")

gc.collect()

start = time.time()
svm = SVC(kernel='linear')
svm.fit(XTrainFeat, yTrainEnc)
svmPred = svm.predict(XTestFeat)
print(f"SVM Accuracy: {accuracy_score(yTestEnc, svmPred):.4f}")
print(f"SVM Time: {time.time() - start:.2f} sec")

gc.collect()

# === CNN training ===
yTrainOneHot = tf.keras.utils.to_categorical(yTrainEnc)
yValidOneHot = tf.keras.utils.to_categorical(yValidEnc)
yTestOneHot = tf.keras.utils.to_categorical(yTestEnc)

cnn = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=XTrain.shape[1:]),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(yTrainOneHot.shape[1], activation='softmax')
])
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnnChk = ModelCheckpoint("cnn_best.keras", save_best_only=True, monitor='val_accuracy', mode='max')

print("Training CNN...")
start = time.time()
cnn.fit(XTrain, yTrainOneHot, epochs=10, validation_data=(XValid, yValidOneHot), batch_size=32)
cnnEval = cnn.evaluate(XTest, yTestOneHot, verbose=0)
print(f"CNN Accuracy: {cnnEval[1]:.4f}")
print(f"CNN Time: {time.time() - start:.2f} sec")

gc.collect()

# === MobileNetV2 fine-tuning ===
XTrainPre = preprocess_input(XTrain.astype(np.float32))
XValidPre = preprocess_input(XValid.astype(np.float32))
XTestPre = preprocess_input(XTest.astype(np.float32))

baseModelFT = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))
baseModelFT.trainable = False

mobilenet = Sequential([
    baseModelFT,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(yTrainOneHot.shape[1], activation='softmax')
])
mobilenet.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
mobileChk = ModelCheckpoint("mobilenet_best.keras", save_best_only=True, monitor='val_accuracy', mode='max')

print("Training MobileNetV2...")
start = time.time()
mobilenet.fit(XTrainPre, yTrainOneHot, epochs=10, validation_data=(XValidPre, yValidOneHot), batch_size=32)
mobileEval = mobilenet.evaluate(XTestPre, yTestOneHot, verbose=0)
print(f"MobileNetV2 Accuracy: {mobileEval[1]:.4f}")
print(f"MobileNetV2 Time: {time.time() - start:.2f} sec")

gc.collect()


Loading data...
Samples: train=14465, valid=1864, test=1874
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Extracting train features...
Extracting valid features...
Extracting test features...
KNN Accuracy: 0.9957
KNN Time: 2.71 sec
SVM Accuracy: 0.9803
SVM Time: 37.81 sec
Training CNN...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 23s 41ms/step - accuracy: 0.5585 - loss: 22.0297 - val_accuracy: 0.7827 - val_loss: 0.4496
Epoch 2/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.8109 - loss: 0.4018 - val_accuracy: 0.9265 - val_loss: 0.1845
Epoch 3/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.9296 - loss: 0.1776 - val_accuracy: 0.9635 - val_loss: 0.0969
Epoch 4/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.9595 - loss: 0.1111 - val_accuracy: 0.9844 - val_loss: 0.0454
Epoch 5/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.9737 - loss: 0.0740 - val_accuracy: 0.9893 - val_loss: 0.0318
Epoch 6/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.9882 - loss: 0.0398 - val_accuracy: 0.9925 - val_loss: 0.0248
Epoch 7/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.9876 - loss: 0.0385 - val_accuracy: 0.9909 - val_loss: 0.0285
Epoch 8/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step - accuracy: 0.9906 - loss: 0.0257 -

1072

In [3]:
import joblib

# After training KNN and SVM
joblib.dump(knn, "knn_model.joblib")
joblib.dump(svm, "svm_model.joblib")

# After training CNN
cnn.save("cnn_best.keras")

# After training MobileNetV2
mobilenet.save("mobilenet_best.keras")


In [22]:
import os, cv2, numpy as np, tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from google.colab import files
import joblib

labelEncoder = LabelEncoder()
labelEncoder.classes_ = np.array(['fractured', 'not fractured'])

knn = joblib.load("knn_model.joblib") if os.path.exists("knn_model.joblib") else None
svm = joblib.load("svm_model.joblib") if os.path.exists("svm_model.joblib") else None
cnn = load_model("cnn_best.keras") if os.path.exists("cnn_best.keras") else None
mobilenet = load_model("mobilenet_best.keras") if os.path.exists("mobilenet_best.keras") else None

mobilenetFeatureExtractor = MobileNetV2(input_shape=(128,128,3), include_top=False, pooling='avg')

def extractFeatures(data):
    imgBatch = np.expand_dims(data, axis=0)
    imgPre = preprocess_input(imgBatch)
    features = mobilenetFeatureExtractor.predict(imgPre)
    return features.reshape(1, -1)

def finalEnsemblePrediction(predictions, weights):
    weightedVotes = {}
    for pred, w in zip(predictions, weights):
        weightedVotes[pred] = weightedVotes.get(pred, 0) + w
    return max(weightedVotes, key=weightedVotes.get)

uploaded = files.upload()
imgPath = list(uploaded.keys())[0]

def predictAllModels(imgPath):
    img = cv2.imread(imgPath)
    img = cv2.resize(img, (128, 128))
    imgArr = np.array(img).astype(np.float32)
    imgBatch = np.expand_dims(imgArr, axis=0)

    preds = []
    print("\n📸 Predictions for:", os.path.basename(imgPath))

    if knn:
        features = extractFeatures(imgArr)  # features shape (1,1280)
        p = knn.predict(features)[0]
        preds.append(p)
        print("KNN:", labelEncoder.inverse_transform([p])[0])

    if svm:
        features = extractFeatures(imgArr)
        p = svm.predict(features)[0]
        preds.append(p)
        print("SVM:", labelEncoder.inverse_transform([p])[0])

    if cnn:
        predProb = cnn.predict(imgBatch, verbose=0)
        p = np.argmax(predProb)
        preds.append(p)
        print("CNN:", labelEncoder.inverse_transform([p])[0])

    if mobilenet:
        imgPre = preprocess_input(imgBatch)
        predProb = mobilenet.predict(imgPre, verbose=0)
        p = np.argmax(predProb)
        preds.append(p)
        print("MobileNetV2:", labelEncoder.inverse_transform([p])[0])

    accs = np.array([0.9957, 0.9803, 0.9840, 0.9931])
    weights = accs / accs.sum()

    finalPred = finalEnsemblePrediction(preds, weights)
    print("🔥 Final Ensemble Prediction:", labelEncoder.inverse_transform([finalPred])[0])

predictAllModels(imgPath)


Saving 6.jpeg to 6 (1).jpeg

📸 Predictions for: 6 (1).jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
KNN: not fractured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
SVM: not fractured
CNN: not fractured
MobileNetV2: fractured
🔥 Final Ensemble Prediction: not fractured
